In [655]:
import functools
from typing import Optional, Sequence, Any, Mapping

import clingo
import clingo.ast
import clingox.backend
import clingox.reify

In [656]:
def solve(ctl: clingo.Control, report=True, report_models=True, report_result=True, sep='\n'):
    models = []
    with ctl.solve(yield_=True) as solve_handle:
        for model in solve_handle:
            symbols = sorted(model.symbols(shown=True))
            if report and report_models:
                print("Answer {}:".format(model.number), end=' ')
                print("{", sep.join(map(str, symbols)), "}", sep=sep)
            models.append(symbols)
        solve_result = solve_handle.get()
        if report and report_result:
            print(solve_result, end='')
            if models:
                print(" {}{}".format(len(models), '' if solve_result.exhausted else '+'))
    return models

In [657]:
p1 = """
a :- k, not b.   b :- not a.   f :- e, not k, not c.
k :- e, not b.   c :- k.       e.
c :- a, b.
"""

In [658]:
class Manipulator(clingo.ast.Transformer):
    loc = clingo.ast.Location(clingo.ast.Position('<string>', 1, 1), clingo.ast.Position('<string>', 1, 1))

In [659]:
class FactToExternal(Manipulator):

    def visit_Rule(self, rule: clingo.ast.AST, external_type: Optional[clingo.ast.AST] = None):
        if not rule.body:
            if external_type is None:
                external_type = clingo.ast.SymbolicTerm(Manipulator.loc, clingo.Function('false'))
            # TODO: check external_type
            return clingo.ast.External(rule.location, rule.head.atom, (), external_type)
        return rule

In [660]:
class BaseCombinator:

    def transform(self, program: str) -> Sequence[clingo.ast.AST]:
        nodes = []
        clingo.ast.parse_string(program, nodes.append)
        return nodes

    def apply(self, program: str, control: Optional[clingo.Control] = None):
        if control is None:
            control = clingo.Control()
        nodes = self.transform(program)
        with clingo.ast.ProgramBuilder(control) as builder:
            for node in nodes:
                builder.add(node)
        return control

In [661]:
class SequenceCombinator(BaseCombinator):

    def __init__(self, manipulators: Sequence[Manipulator], args: Optional[Sequence[Sequence[Any]]] = None,
                 kwargs: Optional[Sequence[Mapping[str, Any]]] = None):
        self._manipulators = manipulators
        self._args = args or tuple(() for _ in manipulators)
        self._kwargs = kwargs or tuple({} for _ in manipulators)

    def _transform_callback(self, stm: clingo.ast.AST, nodes: Optional[Sequence[clingo.ast.AST]] = None) -> None:
        if nodes is None:
            nodes = []
        node = stm
        for i, manipulator in enumerate(self._manipulators):
            args, kwargs = self._args[i], self._kwargs[i]
            node = manipulator.visit(node, *args, **kwargs)
        nodes.append(node)

    def transform(self, program: str) -> Sequence[clingo.ast.AST]:
        nodes = []
        clingo.ast.parse_string(program, functools.partial(self._transform_callback, nodes=nodes))
        return nodes

In [662]:
fact_to_external = SequenceCombinator((FactToExternal(),))
p1t_ast = fact_to_external.transform(p1)
p1t = '\n'.join(map(str, p1t_ast))
print(p1t)

#program base.
a :- k; not b.
b :- not a.
f :- e; not k; not c.
k :- e; not b.
c :- k.
#external e. [false]
c :- a; b.


In [663]:
reified = sorted(clingox.reify.reify_program(p1t))
print('{', '\n'.join(map(str, reified)), '}', sep='\n')

{
atom_tuple(0)
atom_tuple(1)
atom_tuple(2)
atom_tuple(3)
atom_tuple(4)
literal_tuple(0)
literal_tuple(1)
literal_tuple(2)
literal_tuple(3)
literal_tuple(4)
literal_tuple(5)
literal_tuple(6)
literal_tuple(7)
literal_tuple(8)
literal_tuple(9)
literal_tuple(10)
tag(incremental)
atom_tuple(0,2)
atom_tuple(1,4)
atom_tuple(2,3)
atom_tuple(3,5)
atom_tuple(4,6)
external(1,false)
literal_tuple(0,-3)
literal_tuple(1,-2)
literal_tuple(1,1)
literal_tuple(2,-2)
literal_tuple(2,4)
literal_tuple(3,2)
literal_tuple(3,3)
literal_tuple(4,4)
literal_tuple(5,-5)
literal_tuple(5,-4)
literal_tuple(5,1)
literal_tuple(6,2)
literal_tuple(7,3)
literal_tuple(8,5)
literal_tuple(9,1)
literal_tuple(10,6)
output(a,7)
output(b,6)
output(c,8)
output(e,9)
output(f,10)
output(k,4)
rule(disjunction(0),normal(0))
rule(disjunction(1),normal(1))
rule(disjunction(2),normal(2))
rule(disjunction(3),normal(3))
rule(disjunction(3),normal(4))
rule(disjunction(4),normal(5))
}


In [664]:
applicability = """

:- rule(choice(N1), normal(N2)).
:- rule(disjunction(N1), normal(N2)), atom_tuple(N1), 2 { atom_tuple(N1, N3) }.

"""

In [665]:
ctl = clingo.Control()
ctl.add('base', [], applicability)
with clingox.backend.SymbolicBackend(ctl.backend()) as symbolic_backend:
    for symbol in reified:
        symbolic_backend.add_rule(head=(symbol,))
ctl.ground([('base', [])])

In [666]:
solve(ctl, report_models=False);

SAT 1+


In [667]:
ctl = clingo.Control()
ctl.configuration.solve.models = 0
ctl.add('base', [], p1)
ctl.ground([('base', [])])

In [668]:
models = solve(ctl, sep=' ')

Answer 1: { b e f }
Answer 2: { a c e k }
SAT 2


In [669]:
ctl.configuration.solve.enum_mode = 'cautious'

In [670]:
cautious_consequence = solve(ctl, sep=' ')[-1]

Answer 1: { b e f }
Answer 2: { e }
SAT 2


In [671]:
preprocessing = """

head_body(H, ()) :-
  external(A, _),
  output(H, L), literal_tuple(L), literal_tuple(L, A).

head_body(H, ()) :-
  rule(disjunction(D), normal(N)),
  atom_tuple(D), atom_tuple(D, A), output(H, L), literal_tuple(L), literal_tuple(L, A),
  literal_tuple(N), #false: literal_tuple(N, A').

head_body(H, N) :-
  rule(disjunction(D), normal(N)),
  atom_tuple(D), atom_tuple(D, A), output(H, L), literal_tuple(L), literal_tuple(L, A).

body(N, B) :- rule(disjunction(_), normal(N)),
  literal_tuple(N), literal_tuple(N, A),
  literal_tuple(L), literal_tuple(L, A),
  output(B, L).

body(N, -B) :- rule(disjunction(_), normal(N)),
  literal_tuple(N), literal_tuple(N, -A),
  literal_tuple(L), literal_tuple(L, A),
  output(B, L).

e(H, ()) :- head_body(H, ()).
l((), "T").

e(H, N)  :- holds(H), head_body(H, N).
e(-H, N) :- not holds(H), head_body(H, N).
l(N, B)  :- holds(H), head_body(H, N), body(N, B).
l(N, -B) :- holds(H), head_body(H, N), body(N, -B).

l(N, -B) :- not holds(H), head_body(H, N), body(N, B), not holds(B).
l(N, B)  :- not holds(H), head_body(H, N), body(N, -B), holds(B).

#show head_body/2.
#show body/2.
#show e/2.
#show l/2.

"""

In [672]:
model = models[0]

In [673]:
ctl = clingo.Control()
ctl.configuration.solve.models = 0
ctl.add('base', [], preprocessing)
with clingox.backend.SymbolicBackend(ctl.backend()) as symbolic_backend:
    for symbol in reified:
        symbolic_backend.add_rule(head=(symbol,))
    for symbol in model:
        symbolic_backend.add_rule(head=(clingo.Function('holds', (symbol,)),))
ctl.ground([('base', [])])

<block>:38:1-14: info: no atoms over signature occur in program:
  body/2

<block>:39:1-11: info: no atoms over signature occur in program:
  e/2

<block>:37:1-19: info: no atoms over signature occur in program:
  head_body/2

<block>:40:1-11: info: no atoms over signature occur in program:
  l/2



In [674]:
preprocessed = solve(ctl);

Answer 1: {
body(0,-a)
body(1,e)
body(1,-b)
body(2,k)
body(2,-b)
body(3,a)
body(3,b)
body(4,k)
body(5,e)
body(5,-c)
body(5,-k)
e(b,0)
e(e,())
e(f,5)
e(-a,2)
e(-c,3)
e(-c,4)
e(-k,1)
head_body(a,2)
head_body(b,0)
head_body(c,3)
head_body(c,4)
head_body(e,())
head_body(f,5)
head_body(k,1)
l(0,-a)
l(1,b)
l(2,b)
l(2,-k)
l(3,-a)
l(4,-k)
l(5,e)
l(5,-c)
l(5,-k)
l((),"T")
}
SAT 1


In [675]:
preprocessed = """



"""